In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
rng=np.random.RandomState(0)
df=pd.DataFrame({'key':['a','b','c','a','b','c'],
                'data1':range(6),
                'data2':rng.randint(0,10,6),
                },
                columns=['key','data1','data2'])
df

key  data1  data2
0   a      0      5
1   b      1      0
2   c      2      3
3   a      3      3
4   b      4      7
5   c      5      9

In [4]:
import seaborn as sns
planets=sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
planets.head()

method  number  orbital_period   mass  distance  year
0  Radial Velocity       1         269.300   7.10     77.40  2006
1  Radial Velocity       1         874.774   2.21     56.95  2008
2  Radial Velocity       1         763.000   2.60     19.84  2011
3  Radial Velocity       1         326.030  19.40    110.62  2007
4  Radial Velocity       1         516.220  10.50    119.47  2009

In [4]:
planets.dropna().describe()

number  orbital_period        mass    distance         year
count  498.00000      498.000000  498.000000  498.000000   498.000000
mean     1.73494      835.778671    2.509320   52.068213  2007.377510
std      1.17572     1469.128259    3.636274   46.596041     4.167284
min      1.00000        1.328300    0.003600    1.350000  1989.000000
25%      1.00000       38.272250    0.212500   24.497500  2005.000000
50%      1.00000      357.000000    1.245000   39.940000  2009.000000
75%      2.00000      999.600000    2.867500   59.332500  2011.000000
max      6.00000    17337.500000   25.000000  354.000000  2014.000000

# GroupBy对象

### 按列取值

In [5]:
planets.groupby('method')

DataFrameGroupBy

In [6]:
planets.groupby('method')['orbital_period']

SeriesGroupBy object

In [7]:
planets.groupby('method')['orbital_period'].median()
#中位数

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

### 按组迭代

In [8]:
for method,group in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method,group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


### 调用方法

In [9]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.000000
       Eclipse Timing Variations           9.000000
       Imaging                            38.000000
       Microlensing                       23.000000
       Orbital Brightness Modulation       3.000000
       Pulsar Timing                       5.000000
       Pulsation Timing Variations         1.000000
       Radial Velocity                   553.000000
       Transit                           397.000000
       Transit Timing Variations           4.000000
mean   Astrometry                       2011.500000
       Eclipse Timing Variations        2010.000000
       Imaging                          2009.131579
       Microlensing                     2009.782609
       Orbital Brightness Modulation    2011.666667
       Pulsar Timing                    1998.400000
       Pulsation Timing Variations      2007.000000
       Radial Velocity                  2007.518987
       Transit             

### 四大高效操作

#### 累计

In [13]:
df.groupby('key').aggregate(['min',np.median,max])

data1            data2           
      min median max   min median max
key                                  
a       0    1.5   3     3    4.0   5
b       1    2.5   4     0    3.5   7
c       2    3.5   5     3    6.0   9

In [15]:
df.groupby('key').aggregate({'data1':min,
                            'data2':'max',
                            })

data1  data2
key              
a        0      5
b        1      7
c        2      9

#### 过滤（变形）

In [11]:
def func(x):
    return (x['data2'].std()>4)

In [12]:
print(df)
print(df.groupby('key').std())
print(df.groupby('key').filter(func))

  key  data1  data2
0   a      0      5
1   b      1      0
2   c      2      3
3   a      3      3
4   b      4      7
5   c      5      9
       data1     data2
key                   
a    2.12132  1.414214
b    2.12132  4.949747
c    2.12132  4.242641
  key  data1  data2
1   b      1      0
2   c      2      3
4   b      4      7
5   c      5      9


#### 转换（不变形）

In [13]:
df.groupby('key').transform(lambda x:x-x.mean())

data1  data2
0   -1.5    1.0
1   -1.5   -3.5
2   -1.5   -3.0
3    1.5   -1.0
4    1.5    3.5
5    1.5    3.0

#### 应用

In [15]:
def func(x):
    x['data1']/=x['data2'].sum()
    return x

In [16]:
print(df)
print(df.groupby('key').apply(func))

  key  data1  data2
0   a      0      5
1   b      1      0
2   c      2      3
3   a      3      3
4   b      4      7
5   c      5      9
  key     data1  data2
0   a  0.000000      5
1   b  0.142857      0
2   c  0.166667      3
3   a  0.375000      3
4   b  0.571429      7
5   c  0.416667      9


### 设置分割的键

In [31]:
L=[0,1,0,1,2,0]
print(df)
print(df.groupby(L).sum())

  key  data1  data2
0   a      0      5
1   b      1      0
2   c      2      3
3   a      3      3
4   b      4      7
5   c      5      9
   data1  data2
0      7     17
1      4      3
2      4      7


In [32]:
df2=df.set_index('key')
mapping={'a':'vowel','b':'bbb','c':'bbb'}
print(df2)
print(df2.groupby(mapping).sum())

     data1  data2
key              
a        0      5
b        1      0
c        2      3
a        3      3
b        4      7
c        5      9
       data1  data2
bbb       12     19
vowel      3      8


In [34]:
df2.groupby(str.upper).mean()

data1  data2
A    1.5    4.0
B    2.5    3.5
C    3.5    6.0

In [37]:
df2.groupby([str.upper,mapping]).mean()

data1  data2
A vowel    1.5    4.0
B bbb      2.5    3.5
C bbb      3.5    6.0